# Přijímací zkouška

Uvažujme nyní vyhodnocení přijímací zkoušky na střední školu technického zaměření, které jsou uloženy v souboru [prijimaci_zkousky.csv](prijimaci_zkousky.csv). 

Uchazeči a uchazečky musejí absolvovat tři písemné testy: test z anglického jazyka, matematiky a českého jazyka. 

Pokud dostane z některého z testů méně než 60 bodů, dostane hodnocení 0. 

Pokud má ze všech testů více než 60 bodů, je jeho skóre rovno součtu bodů ze všech tří testů. 


Dále uvažujeme, že za aktivity uchazeče přičítáme bonusové body:

- za 1. až 10. místo v některé z olympiád 10 bonusových bodů,
- za účast na letní škole dostane 5 bonusových bodů.


Níže jsou data. Informace o olympiádách jsou ve sloupci `souteze`. Jsou tam uvedeny pouze údaje, které se započítávají do zkoušky, tj. získání 1. až 10. místa.

In [2]:
import pandas

data = pandas.read_csv("prijimaci_zkousky.csv")
data.head()

,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze
0,Květoslava,Růžičková,Elektrotechnika,62,48,51,ne,NaN
1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda)
2,Denis,Fiala,Informatika,63,69,65,ne,NaN
3,Luboš,Šimek,Informatika,78,79,65,ne,NaN
4,Vilém,Novotný,Informatika,84,68,78,ano,NaN


Vytvoř funkci `get_points()`, která přidělí každému z uchazečů počet bodů na základě daných kritérií. 

In [7]:
def get_points(row):
    if row[["body_aj", "body_mat", "body_cj"]].min() < 60:
        return 0
    skore = row[["body_aj", "body_mat", "body_cj"]].sum()
    if not pandas.isnull(row["souteze"]):
        skore = skore + 10
    if row["letni_skola"] == "ano":
        skore = skore + 5
    return skore
    
    
data["body"] = data.apply(get_points, axis=1)
data.head()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi,prijat
10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0,Ano
89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0,Ano
8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0,Ano
98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),270,3.0,Ano
45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0,Ano


Dále vytvoř novou tabulku, která obsahuje pouze uchazeče a uchazečky, kteří mají nenulové hodnocení. 

In [4]:
data = data[data["body"] > 0]
data = data.reset_index()
data.head()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body
0,1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda),231
1,2,Denis,Fiala,Informatika,63,69,65,ne,NaN,197
2,3,Luboš,Šimek,Informatika,78,79,65,ne,NaN,222
3,4,Vilém,Novotný,Informatika,84,68,78,ano,NaN,235
4,5,Drahomíra,Štěpánková,Elektrotechnika,69,74,60,ne,NaN,203


Dále využij metodu `rank()` k výpočtu pořadí uchazeče/uchazečky v rámci daného oboru, a data seřaď podle oboru a počtu bodů. 

In [5]:
data["poradi"] = data.groupby("obor")["body"].rank(method="min", ascending=False)
data = data.sort_values(["obor", "poradi"])
data.head()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi
10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0
89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0
8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0
98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),270,3.0
45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0


Nakonec vytvoř tabulku s přijatými uchazeči, kde budou uchazeči hodnocení od 1. do 30. místa pro daný obor.

In [6]:
def prijat(row):
    if row["poradi"] <= 30:
        return "Ano"
    else:
        return "Ne"

data["prijat"] = data.apply(prijat, axis=1)
data

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi,prijat
10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0,Ano
89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0,Ano
8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0,Ano
98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),270,3.0,Ano
45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0,Ano
...,...,...,...,...,...,...,...,...,...,...,...,...
54,87,Veronika,Poláková,Technické lyceum,68,68,61,ano,NaN,202,27.0,Ano
25,38,Monika,Kratochvílová,Technické lyceum,60,76,65,ne,NaN,201,28.0,Ano
11,15,Erik,Dvořák,Technické lyceum,68,71,61,ne,NaN,200,29.0,Ano
119,195,Naděžda,Čermáková,Technické lyceum,71,61,67,ne,NaN,199,30.0,Ano


### Bonusy

- Zájemcům o studium na oborech Elektrotechnika a Informatika přiděluj bonusové body pouze za fyzikální nebo matematickou olympiádu, zájemcům o lyceum i za ostatní olympiády.
- Za 1. až 3. místo v olympiádě přiděluj 15 bodů, za nižší místa 5.

In [22]:
import re

def get_points(row):
    if row[["body_aj", "body_mat", "body_cj"]].min() < 60:
        return 0
    skore = row[["body_aj", "body_mat", "body_cj"]].sum()
    if not pandas.isnull(row["souteze"]):
        obor = row["obor"]
        if obor in ["Elektrotechnika", "Informatika"]:
            if "fyzikální" in row["souteze"] or "matematická" in row["souteze"]:
                if re.match("^[123]", row["souteze"]):
                # if row["souteze"].startswith("1.") or row["souteze"].startswith("2.") or row["souteze"].startswith("3."):
                    skore += 15
                else:
                    skore += 5
        else:
            if re.match("^[123]", row["souteze"]):
                skore += 15
            else:
                skore += 5
    if row["letni_skola"] == "ano":
        skore += 5
    return skore


data["body"] = data.apply(get_points, axis=1)
data.head()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi,prijat
10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0,Ano
89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0,Ano
8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0,Ano
98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),260,3.0,Ano
45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0,Ano


---
---

### Nápověda

Níže jsou data včetně výpočtu bodů.

In [10]:
data = pandas.read_csv("prijimaci_zkouska_body.csv")
data.head(15)

,Unnamed: 0,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body
0,0,Květoslava,Růžičková,Elektrotechnika,62,48,51,ne,NaN,0
1,1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda),231
2,2,Denis,Fiala,Informatika,63,69,65,ne,NaN,197
3,3,Luboš,Šimek,Informatika,78,79,65,ne,NaN,222
4,4,Vilém,Novotný,Informatika,84,68,78,ano,NaN,235
5,5,Drahomíra,Štěpánková,Elektrotechnika,69,74,60,ne,NaN,203
6,6,Vlasta,Horáková,Technické lyceum,53,61,57,ne,NaN,0
7,7,Šimon,Pospíšil,Technické lyceum,70,66,68,ne,NaN,204
8,8,Jaroslav,Říha,Elektrotechnika,79,71,63,ne,NaN,213
9,9,Renata,Malá,Elektrotechnika,71,66,75,ano,NaN,217


Níže jsou data včetně pořadí (po vyřazení uchazečů, kteří měli jeden z testů na méně než 60 bodů).

In [11]:
data = pandas.read_csv("prijimaci_zkouska_poradi.csv")
data.head(15)

,Unnamed: 0,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi
0,10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0
1,89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0
2,8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0
3,98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),270,3.0
4,45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0
5,85,138,Anežka,Kolářová,Elektrotechnika,92,78,91,ne,NaN,261,6.0
6,103,166,Radek,Bláha,Elektrotechnika,103,73,83,ne,NaN,259,7.0
7,78,123,Karolína,Nováková,Elektrotechnika,84,95,79,ne,NaN,258,8.0
8,115,186,Vilém,Marek,Elektrotechnika,92,85,79,ne,NaN,256,9.0
9,74,118,Erik,Beneš,Elektrotechnika,73,87,86,ano,NaN,251,10.0


Níže jsou data včetně rozhodnutí o přijetí.

In [12]:
data = pandas.read_csv("prijimaci_zkouska_rozhodnuti.csv")
data.head()

,Unnamed: 0,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi,prijat
0,10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0,Ano
1,89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0,Ano
2,8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0,Ano
3,98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),270,3.0,Ano
4,45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0,Ano


In [13]:
data = pandas.read_csv("prijimaci_zkouska_rozhodnuti.csv")
data.tail()

,Unnamed: 0,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi,prijat
117,54,87,Veronika,Poláková,Technické lyceum,68,68,61,ano,NaN,202,27.0,Ano
118,25,38,Monika,Kratochvílová,Technické lyceum,60,76,65,ne,NaN,201,28.0,Ano
119,11,15,Erik,Dvořák,Technické lyceum,68,71,61,ne,NaN,200,29.0,Ano
120,119,195,Naděžda,Čermáková,Technické lyceum,71,61,67,ne,NaN,199,30.0,Ano
121,21,31,Jan,Kadlec,Technické lyceum,61,67,62,ne,NaN,190,31.0,Ne
